In [4]:
import pandas as pd

df = pd.read_csv('FD_NAIS_2021.csv', sep = ';')

df.head()

C:\Users\MANEL\AppData\Local\Temp\ipykernel_12996\3088685770.py:3: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('FD_NAIS_2021.csv', sep = ';')


,ANAIS,ACCOUCHR,AGEMERE,AGEPERE,AGEXACTM,AGEXACTP,AMAR,ARECC,ARECM,ARECP,...,MRECC,MRECM,MRECP,NBENF,ORIGINOM,SEXE,SITUATMR,SITUATPR,TUCOM,TUDOM
0,2021,ES,31,34,31,34,2020,0,0,0,...,0,0,0,1,1.0,1,S,S,NaN,9
1,2021,ES,32,32,32,31,0,0,2021,0,...,0,8,0,1,1.0,2,S,ND,NaN,9
2,2021,ES,25,35,25,34,2018,0,0,0,...,0,0,0,1,1.0,1,S,S,NaN,9
3,2021,ES,28,39,27,38,2018,0,0,0,...,0,0,0,1,1.0,1,ND,ND,NaN,9
4,2021,ES,39,38,39,38,2011,0,0,0,...,0,0,0,1,1.0,1,ND,ND,NaN,9


In [5]:
import pandas as pd

df = pd.read_csv('DS_EC_NAIS_metadata.csv', sep = ';')

print(df.shape)

(208, 4)


In [11]:
import pandas as pd

df = pd.read_csv('DS_EC_DECES_data.csv', sep = ';')
df.head(10)

C:\Users\MANEL\AppData\Local\Temp\ipykernel_15000\2556967555.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('DS_EC_DECES_data.csv', sep = ';')


,GEO,GEO_OBJECT,SEX,AGE,DTH_PLACE,FREQ,EC_MEASURE,OBS_STATUS_FR,TIME_PERIOD,OBS_VALUE
0,55,DEP,_T,_T,_T,A,DTH,D,2019,2148.0
1,99,OTHER,M,Y50T64,_T,D,DTH,D,2024-08-17,1.0
2,99,OTHER,M,Y75T84,_T,D,DTH,PROV,2025-08-02,4.0
3,99,OTHER,M,Y_GE85,_T,D,DTH,PROV,2025-08-02,3.0
4,99,OTHER,F,Y75T84,_T,D,DTH,D,2024-05-01,1.0
5,99,OTHER,F,Y65T74,_T,D,DTH,D,2024-01-06,1.0
6,99,OTHER,M,Y50T64,_T,D,DTH,D,2024-12-13,2.0
7,26,DEP,_T,_T,_T,A,DTH,D,2024,5154.0
8,05,DEP,_T,_T,_T,A,DTH,D,2017,1388.0
9,44,DEP,_T,_T,_T,M,DTH,PROV,2025-05,979.0
